In [ ]:
import requests
import pandas as pd 
import time
import numpy as np
import os
from io import StringIO
from custom_function import *


# define request header
headers = {'user-agent': os.environ.get("ONS-USER-AGENT")}

## 1. Explore the dataset available



In [37]:
# make API requests
url = "https://api.beta.ons.gov.uk/v1/datasets"
res = requests_get(url,headers)

# parse into a pd DataFrame
df = pd.json_normalize(res.json()['items'])



requesting https://api.beta.ons.gov.uk/v1/datasets


In [38]:
# parse the metadata
meta_df = df[['id','description','links.latest_version.href']]\
            .rename(columns={"links.latest_version.href":"href"})

In [44]:
# parse the metadata
meta_df = df[['id','description','links.latest_version.href']]\
            .rename(columns={"links.latest_version.href":"href"})

# instantiate empty df
dataset_df = pd.DataFrame()
for key, row in meta_df.iterrows():
    res = requests_get(row[-1],headers)
    dataset = pd.json_normalize(res.json()['downloads'])
    dataset['id'] = row['id']
    # append to the df
    dataset_df = pd.concat([dataset_df, dataset], ignore_index = True)
    time.sleep(0.5)


requesting https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly/editions/time-series/versions/9
requesting https://api.beta.ons.gov.uk/v1/datasets/wellbeing-local-authority/editions/time-series/versions/4
requesting https://api.beta.ons.gov.uk/v1/datasets/weekly-deaths-region/editions/covid-19/versions/170
requesting https://api.beta.ons.gov.uk/v1/datasets/weekly-deaths-local-authority/editions/2023/versions/50
requesting https://api.beta.ons.gov.uk/v1/datasets/weekly-deaths-health-board/editions/2023/versions/50
requesting https://api.beta.ons.gov.uk/v1/datasets/weekly-deaths-age-sex/editions/covid-19/versions/170
requesting https://api.beta.ons.gov.uk/v1/datasets/uk-spending-on-cards/editions/time-series/versions/112
requesting https://api.beta.ons.gov.uk/v1/datasets/uk-business-by-enterprises-and-local-units/editions/2022/versions/1
requesting https://api.beta.ons.gov.uk/v1/datasets/traffic-camera-activity/editions/time-series/versions/88
requesting https://api.beta.ons.gov.u

In [131]:
dataset_df.head()

,csv.href,csv.size,csvw.href,csvw.size,xls.href,xls.size,id
0,https://download.beta.ons.gov.uk/downloads/dat...,265571,https://download.beta.ons.gov.uk/downloads/dat...,2779,https://download.beta.ons.gov.uk/downloads/dat...,33265,wellbeing-quarterly
1,https://download.beta.ons.gov.uk/downloads/dat...,9515176,https://download.beta.ons.gov.uk/downloads/dat...,3542,https://download.beta.ons.gov.uk/downloads/dat...,2250646,wellbeing-local-authority
2,https://download.beta.ons.gov.uk/downloads/dat...,772791,https://download.beta.ons.gov.uk/downloads/dat...,1830,https://download.beta.ons.gov.uk/downloads/dat...,86858,weekly-deaths-region
3,https://download.beta.ons.gov.uk/downloads/dat...,49325723,https://download.beta.ons.gov.uk/downloads/dat...,2124,https://download.beta.ons.gov.uk/downloads/dat...,10837647,weekly-deaths-local-authority
4,https://download.beta.ons.gov.uk/downloads/dat...,1262931,https://download.beta.ons.gov.uk/downloads/dat...,2020,https://download.beta.ons.gov.uk/downloads/dat...,241893,weekly-deaths-health-board


## 2. Download the CSV File

In [162]:
for key, value in dataset_df.iterrows():
    if str(value['csv.href']) != "nan":
        # storage_options = {'User-Agent': 'Mozilla/5.0'}
        df = pd.read_csv(value['csv.href'],
                    storage_options=headers)
        df.to_csv(f"./data/{value['id']}.csv")
        time.sleep(5)
